## Importação das Bibiliotecas

In [2]:
%run C:/Users/leoba/Documents/Enem/Notebooks/PYSpark/Spark_Run.ipynb

Caminho atual: c:\Users\leoba\Documents\Enem\Notebooks\PYSpark\ModeloAvro
Caminho até 'Enem/': c:\Users\leoba\Documents\Enem


In [3]:
import pyspark.sql.functions as F


# Testando configuração arquivo Parquet


In [4]:
file_csv = r'C:\Users\leoba\Documents\Enem\Dataset\Model\MICRODADOS_ENEM_MODEL.CSV'

#Função Paa ler os arquivos CSV
def lerCSV(spark, caminho_arquivo):
    df = spark.read.csv(caminho_arquivo, sep=';', header=True, inferSchema=True, encoding='ISO-8859-1')
    colunas_selecionadas = [F.col(df.columns[i]) for i in range(0, 13)] + [F.col(df.columns[i]) for i in range(23, 27)]  + [F.col(df.columns[i]) for i in range(31, 35)] + [F.col(df.columns[39])] + [F.col(df.columns[56])]
    return df.select(colunas_selecionadas)



In [5]:
dfDadosEnem = lerCSV(spark, file_csv)


In [6]:
dfDadosEnem.write.csv(path='C:/Users/leoba/Documents/Enem/Dataset/csv', mode='overwrite')


Py4JJavaError: An error occurred while calling o65.csv.
: org.apache.spark.SparkException: Job aborted.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.jobAbortedError(QueryExecutionErrors.scala:496)
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.write(FileFormatWriter.scala:261)
	at org.apache.spark.sql.execution.datasources.InsertIntoHadoopFsRelationCommand.run(InsertIntoHadoopFsRelationCommand.scala:186)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult$lzycompute(commands.scala:113)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult(commands.scala:111)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.executeCollect(commands.scala:125)
	at org.apache.spark.sql.execution.QueryExecution$$anonfun$eagerlyExecuteCommands$1.$anonfun$applyOrElse$1(QueryExecution.scala:97)
	at org.apache.spark.sql.execution.SQLExecution$.$anonfun$withNewExecutionId$5(SQLExecution.scala:103)
	at

In [12]:
dfDadosEnem.write.csv(path='C:/Users/leoba/Documents/Enem/Dataset/csv', mode='overwrite')

Py4JJavaError: An error occurred while calling o106.csv.
: org.apache.spark.SparkException: Job aborted.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.jobAbortedError(QueryExecutionErrors.scala:496)
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.write(FileFormatWriter.scala:261)
	at org.apache.spark.sql.execution.datasources.InsertIntoHadoopFsRelationCommand.run(InsertIntoHadoopFsRelationCommand.scala:186)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult$lzycompute(commands.scala:113)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult(commands.scala:111)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.executeCollect(commands.scala:125)
	at org.apache.spark.sql.execution.QueryExecution$$anonfun$eagerlyExecuteCommands$1.$anonfun$applyOrElse$1(QueryExecution.scala:97)
	at org.apache.spark.sql.execution.SQLExecution$.$anonfun$withNewExecutionId$5(SQLExecution.scala:103)
	a

# Cria arquivo parquet, através do DF Carregado

## Ler arquivo Avro recem criado

## Criando uma SparkSession

## Processo de Leitura do CSV de Participantes
